In [1]:
import numpy as np
import rnn_utils

## 1 - 循环神经网络的前向传播
 我们来看一下下面的循环神经网络的图，在这里使用的是$T_x = T_y$，我们来实现它。
 
![](https://img-blog.csdn.net/20180702215837813?watermark/2/text/aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3UwMTM3MzMzMjY=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

<p style="text-align:center">
**图 1**: 基本的RNN模型
</p>

我们怎么才能实现它呢？有以下步骤：
1. 实现RNN的一个时间步所需要计算的东西。
2.在$Tx$时间步上实现一个循环，以便一次处理所有输入。

1.1 - RNN单元
 循环神经网络可以看作是单元的重复，首先要实现单个时间步的计算，下图描述了RNN单元的单个时间步的操作。
![RNN single step](./images/RNN-single-step.png)
<p style="text-align:center">
**图 2**: 基本的RNN单元.
</p>
输入 $x^{\langle t \rangle}$ (当前输入) 与 $a^{\langle t - 1\rangle}$ (包含过去信息的上一隐藏层的激活值), 输出 $a^{\langle t \rangle}$ 给下一个RNN单元，也用于预测 $y^{\langle t \rangle}$
 现在我们要根据图2来实现一个RNN单元，这需要由以下几步完成：

使用tanh函数计算隐藏单元的激活值：$a^{<t>}=tanh(W_{aa}a^{<t−1>}+Waxx⟨t⟩+ba) a^{\langle t \rangle} = \tanh(W_{aa}a^{\langle t - 1 \rangle} + W_{ax}x^{\langle t \rangle} + b_a)a 
⟨t⟩=tanh(W_{aa}a^<t-1> 
⟨t−1⟩
 +W 
ax
​	
 x 
⟨t⟩
 +b 
a
​	
 )

使用a⟨t⟩ a^{\langle t \rangle}a 
⟨t⟩
 计算yˆ⟨t⟩=softmax(Wyaa⟨t⟩+by) \hat y ^{\langle t \rangle} = softmax( W_{ya}a^{\langle t \rangle} + b_y ) 
y
^
​	
  
⟨t⟩
 =softmax(W 
ya
​	
 a 
⟨t⟩
 +b 
y
​	
 )，softmax在rnn_utils内。

把(a⟨t⟩,a⟨t⟩−1,x⟨t⟩,parameters) ( a^{\langle t \rangle},a^{\langle t \rangle - 1},x^{\langle t \rangle},parameters)(a 
⟨t⟩
 ,a 
⟨t⟩−1
 ,x 
⟨t⟩
 ,parameters)存储到cache中。

返回a⟨t⟩,y⟨t⟩ a^{\langle t \rangle},y^{\langle t \rangle}a 
⟨t⟩
 ,y 
⟨t⟩
 与cache。

 我们将向量化m mm个样本，因此，x⟨t⟩ x^{\langle t \rangle}x 
⟨t⟩
 的维度为(nx,m) (n_x,m)(n 
x
​	
 ,m)，a⟨t⟩ a^{\langle t \rangle}a 
⟨t⟩
 的维度为(na,m) (n_a,m)(n 
a
​	
 ,m)。

In [2]:
def rnn_cell_forward(xt, a_prev, parameters):
    """
    根据图2实现RNN单元的单步前向传播
    
    参数：
        xt -- 时间步“t”输入的数据，维度为（n_x, m）
        a_prev -- 时间步“t - 1”的隐藏隐藏状态，维度为（n_a, m）
        parameters -- 字典，包含了以下内容:
                        Wax -- 矩阵，输入乘以权重，维度为（n_a, n_x）
                        Waa -- 矩阵，隐藏状态乘以权重，维度为（n_a, n_a）
                        Wya -- 矩阵，隐藏状态与输出相关的权重矩阵，维度为（n_y, n_a）
                        ba  -- 偏置，维度为（n_a, 1）
                        by  -- 偏置，隐藏状态与输出相关的偏置，维度为（n_y, 1）
    
    返回：
        a_next -- 下一个隐藏状态，维度为（n_a， m）
        yt_pred -- 在时间步“t”的预测，维度为（n_y， m）
        cache -- 反向传播需要的元组，包含了(a_next, a_prev, xt, parameters)
    """
    
    # 从“parameters”获取参数
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    # 使用上面的公式计算下一个激活值
    a_next = np.tanh(np.dot(Waa, a_prev) + np.dot(Wax, xt) + ba)
    
    # 使用上面的公式计算当前单元的输出
    yt_pred = rnn_utils.softmax(np.dot(Wya, a_next) + by)
    
    # 保存反向传播需要的值
    cache = (a_next, a_prev, xt, parameters)
    
    return a_next, yt_pred, cache

In [5]:
np.random.seed(1)
xt = np.random.randn(3,10)
a_prev = np.random.randn(5,10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a_next, yt_pred, cache = rnn_cell_forward(xt, a_prev, parameters)
print("a_next[4] = ", a_next[4])
print("a_next.shape = ", a_next.shape)
print("yt_pred[1] =", yt_pred[1])
print("yt_pred.shape = ", yt_pred.shape)


a_next[4] =  [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
a_next.shape =  (5, 10)
yt_pred[1] = [0.9888161  0.01682021 0.21140899 0.36817467 0.98988387 0.88945212
 0.36920224 0.9966312  0.9982559  0.17746526]
yt_pred.shape =  (2, 10)
